In [1]:
from pulp import *
import os
import numpy as np
import pandas as pd

def read_matrix(path_file):
    satisfactions = []
    resources = []
    capacities = []
    
    with open(path_file, 'r') as f:
        m, n= f.readline().split(' ')[:2]
        m, n = int(m), int(n)
        lines = f.readlines()
        lines = [line.rstrip().split(' ') for line in lines]

    
        for i in range(0, m):
            for j in range(n):
                satisfactions.append(int(lines[i][j]))


        for i in range(m, m + m):
            for j in range(n):
                resources.append(int(lines[i][j]))
        
        for j in range(m):
            capacities.append(int(lines[2*m][j]))
            
    satisfactions = np.array(satisfactions)
    resources = np.array(resources)
    capacities = np.array(capacities)
    
    return m, n, satisfactions, resources, capacities

In [7]:
inputPath = "./input/d201600.in"

In [59]:
ans = []

solver = CPLEX_CMD(options = [("TimeLimit",180),("Presolve",0),("Cuts",0),("BranchDir", -1)])

prob = LpProblem("desig_gen_problem", LpMaximize)

m, n, sat, res, cap = read_matrix(inputPath)

N = range(n*m)

x = LpVariable.dicts(
    "x", N, lowBound=0, upBound=1.0
)

prob += lpSum([sat[i] * x[i]  for i in N]), 'objective F'

#   every job can only be picked by one worker
for i in range(n):
    prob += lpSum([x[i + j*n] for j in range(m)]) == 1

#   every worker has to take only the jobs that they can
for i in range(m):
    prob += lpSum([x[i*n + j] * res[i*n + j] for j in range(n)]) <= cap[i]

solver.actualSolve(prob)
solver.solverModel.parameters.timelimit.set(10)
solver.solverModel.parameters.preprocessing.presolve.set(0)
solver.solverModel.parameters.mip.strategy.branch.set(1)
print(solver.solverModel.parameters.get_all())

# solver.callSolver(prob)
status = solver.findSolutionValues(prob)



# # df_res = pd.DataFrame(res)
# # df_res.to_csv('results_lp.csv')

TypeError: can only concatenate tuple (not "str") to tuple